In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2025-02-13 20:43:08.789651: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-13 20:43:08.798340: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739459588.808679   45159 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739459588.811742   45159 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-13 20:43:08.822549: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
import os

In [3]:
data_path=os.path.join("..","Data","lstm_model_data_v1.csv")

In [4]:
data=pd.read_csv(data_path)

In [8]:
data.dropna(inplace=True)

In [9]:
data

,processed_text_upto_300_words,label
0,students faculty georgetown law school gathere...,1
1,beirut parliament group lebanon hezbollah said...,1
2,kabul us ambassador afghanistan said monday wo...,1
3,berlin chancellor angela merkel friday said ge...,1
4,easy blame white man way accountable actions l...,0
...,...,...
44892,donald trump literally used national prayer br...,0
44893,time drag donald trump office act soon constit...,0
44895,donald trump misogynistic behavior towards wom...,0
44896,president donald trump declined friday explain...,1


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44196 entries, 0 to 44897
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   processed_text_upto_300_words  44196 non-null  object
 1   label                          44196 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [28]:
data_X=data["processed_text_upto_300_words"]
data_Y=data["label"]

## Tokenize Text

In [163]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [164]:
tokenizer=Tokenizer()

In [165]:
tokenizer.fit_on_texts(data_X)

In [166]:
tokenized_text=tokenizer.texts_to_sequences(data_X)

In [167]:
vocab_length=len(tokenizer.word_index)

In [168]:
vocab_length

108786

In [169]:
word_index=tokenizer.word_index

## Pad Sequences

In [170]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [171]:
padded_text=pad_sequences(tokenized_text,maxlen=300,truncating="post",padding="post")

In [172]:
padded_text.shape

(44196, 300)

## Embedding matrix

In [52]:
embedding_matrix=np.zeros((vocab_length+1,100))

In [53]:
embedding_path=os.path.join("..","Data","glove.6B.100d.txt")

In [59]:
embed_dict={}
with open(embedding_path,'r',encoding='utf-8') as file:
    for line in file:
        vector=line.split()
        word=vector[0]
        embed=np.asarray(vector[1:])
        embed_dict[word]=embed

for word,index in word_index.items():
    if word in embed_dict.keys():
        embedding_matrix[index]=embed_dict[word]
        

In [77]:
embedding_matrix.shape

(108787, 100)

### Important Note: There will be 6 Lstm models and each having 60 words per instance.

In [173]:
from tensorflow.keras.layers import Input


In [174]:
inp=Input(shape=(300,),dtype=tf.int32)

In [175]:
inp.shape

(None, 300)